In [ ]:
import osmnx as ox
import networkx as nx
import time
from shapely.geometry import Polygon
from shapely.geometry import shape
import pandas as pd
import os
import fiona
import numpy as np

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')

    with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
        gdf_nodes.to_file(filepath_nodes, encoding=encoding)
        gdf_edges.to_file(filepath_edges, encoding=encoding)

# San Francisco
bounds = (-122.4365, -122.3859, 37.8140, 37.7673)
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./data')
print("--- %s seconds ---" % (time.time() - start_time))

gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
gdf_nodes.to_csv('.data/SanFrancisco_nodes.csv')
gdf_edges.to_csv('.data/SanFrancisco_edges.csv')